# Submission Volume Prediction: A Comparative Analysis

This notebook walks through the process of forecasting submission volumes using various time series models, including **Prophet**, **LSTM**, and **hybrid approaches combined with Bayesian Model Averaging (BMA)**.  
The goal is to identify the most accurate model for this specific dataset.

---

## 1. Setup: Importing Libraries and Loading Data

**Explanation:**  
This first block imports all necessary Python libraries for:
- Data manipulation
- Time series modeling
- Neural networks
- Evaluation

It also defines the path to the input CSV file and loads the dataset into a `pandas` DataFrame.  
Basic error handling is included to ensure the file is found.


In [ ]:
import pandas as pd
import numpy as np
import os
# import matplotlib.pyplot as plt # Plotting is currently commented out
# import seaborn as sns # Plotting is currently commented out
from prophet import Prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.regularizers import l2

# Load the cleaned submission data exported from SQL
CSV_PATH = "c:/Users/t-matasert/OneDrive - Microsoft/desktop/L7 Data Science & AI/Project/VolumePredictionProject/data/input/SQL_Results_Submissions_Prophet_v3.csv"

# Read CSV and confirm it loaded properly
if os.path.exists(CSV_PATH):
    df = pd.read_csv(CSV_PATH)
    print("Original Data Head:")
    print(df.head(10))
else:
    raise FileNotFoundError(f"CSV file not found at: {CSV_PATH}")

## 2. Initial Data Preprocessing

This block focuses on cleaning the raw data. It involves:

- Stripping any leading/trailing whitespace from column names.
- Converting the `SubmissionCreatedDate` column to datetime objects, ensuring correct parsing of **day-first** dates.
- Standardizing the `BypassFlag` based on the `IsBypassed` column, mapping `'bypass'` and `'bypass once released'` to `1` and others to `0`.
- Filtering out rows where `BypassFlag` is `1` (i.e., keeping only non-bypassed submissions).
- Excluding specified `SubmissionType` categories that are not relevant for this volume prediction task.


In [ ]:
# Clean column names
df.columns = df.columns.str.strip()

# Convert date column, handling potential dayfirst format
df['SubmissionCreatedDate'] = pd.to_datetime(df['SubmissionCreatedDate'], dayfirst=True)

# Update the BypassFlag logic to include 'bypass once released' as bypass
df['BypassFlag'] = df['IsBypassed'].apply(lambda x: 1 if str(x).lower() in ['bypass', 'bypass once released'] else 0)

# Filter out rows where BypassFlag is 1
df = df[df['BypassFlag'] == 0]

# Filter out unnecessary SubmissionTypes
submission_types_to_exclude_prophet = ['Handheld Verified','Stub', 'Canary File', 'Hub App (General)', 'Closed Beta - Not Tested',
                                     'Other', 'Compilation Disc', 'External Beta', 'Internal Beta', 'Beyond Console', 'Open Beta', 'Mouse & Keyboard']
df = df[~df['SubmissionType'].astype(str).str.lower().isin([st.lower() for st in submission_types_to_exclude_prophet])]

print("\nData shape after initial preprocessing:", df.shape)
print("Processed Data Head:")
print(df.head())

## 3. Data Aggregation and Feature Engineering for Time Series

The preprocessed data is now aggregated to a **monthly frequency**. Key steps include:

- Resampling the data by month (`'M'`) based on `SubmissionCreatedDate`.
- Calculating the **distinct count of `SubmissionID`** for each month to get the **monthly submission volume**. This becomes our target variable `y`.
- Storing the original `y` values before transformation for later comparison.
- Applying a **log transformation** (`np.log1p`) to the target variable `y`. This helps stabilize variance and often improves model performance for time series with **exponential trends**.
- Calculating the **average `PackageCount`** per month and adding it as an **exogenous regressor** (`avg_package_count`) to the monthly DataFrame.
- Any missing values in this regressor are **forward-filled** and then **backward-filled**.


In [ ]:
# Aggregate data monthly: count distinct SubmissionID per month
monthly_df = df.resample('M', on='SubmissionCreatedDate')['SubmissionID'].nunique().reset_index(name='y')
monthly_df.columns = ['ds', 'y'] # Prophet requires 'ds' for date and 'y' for target

# Store original 'y' for later comparison and log transform the target variable 'y'
monthly_df['y_original'] = monthly_df['y']
monthly_df['y'] = np.log1p(monthly_df['y'])

# Add average PackageCount per month as a regressor
monthly_avg_package = df.resample('M', on='SubmissionCreatedDate')['PackageCount'].mean().reset_index()
monthly_avg_package.columns = ['ds', 'avg_package_count']
monthly_df = pd.merge(monthly_df, monthly_avg_package, on='ds', how='left')

# Fill NaNs in avg_package_count that might arise from resampling, then ffill/bfill
monthly_df['avg_package_count'] = monthly_df['avg_package_count'].fillna(method='ffill').fillna(method='bfill')

print("\nMonthly Aggregated Data Head:")
print(monthly_df.head())
print("\nMonthly Aggregated Data Info:")
monthly_df.info()

## 4. Defining Holiday Effects

Prophet allows for the inclusion of custom **holiday effects**.  
This block defines a `DataFrame` specifying **"year-end slowdown"** periods around **Christmas** and **New Year**.

For each holiday instance:
- A **lower window** and an **upper window** are defined.
- These windows capture the effects **leading up to and following** the specific holiday dates.


In [ ]:
# Create holiday slowdown dataframe for Prophet
holiday_events = pd.DataFrame({
    'holiday': 'year_end_slowdown',
    'ds': pd.to_datetime([
        '2021-12-24', '2021-12-25', '2022-01-01',
        '2022-12-24', '2022-12-25', '2023-01-01',
        '2023-12-24', '2023-12-25', '2024-01-01',
        '2024-12-24', '2024-12-25', '2025-01-01' # Include future holidays for forecasting
    ]),
    'lower_window': -3, # Affects 3 days before
    'upper_window': 3  # Affects 3 days after
})
print("\nHoliday Events DataFrame:")
print(holiday_events.head())

## 5. Defining Training Cutoff Date

**Explanation:**  
A `cutoff_date` is defined to separate **historical data** used for training the models from the data used for **evaluation** (the "future").

- All models will be trained on data **before** this date.


In [ ]:
# Define train/test split for Prophet and training cutoff for LSTMs
cutoff_date = pd.to_datetime('2024-05-01') # Marks end of training for Prophet and LSTMs
train_monthly_prophet = monthly_df[monthly_df['ds'] < cutoff_date].copy()

print(f"\nData up to {cutoff_date.date()} will be used for training the initial models.")
print(f"Prophet training data shape: {train_monthly_prophet.shape}")

## 6. Prophet Model (Model P) – Initialization and Training

This block initializes the main **Prophet model** (referred to as **Model P**).

- It uses pre-determined **best hyperparameters** for:
  - `changepoint_prior_scale`
  - `seasonality_mode`
  - `seasonality_prior_scale`
  - `holidays_prior_scale`
- **Yearly seasonality** is enabled.
- **Weekly seasonality** is disabled (since the data is monthly).
- `avg_package_count` is added as an **external regressor**.
- A custom **monthly seasonality** is also added.
- The model is then **trained (fit)** using the `train_monthly_prophet` DataFrame, which includes:
  - The **log-transformed target `y`**
  - The **`avg_package_count` regressor**


In [ ]:
# --- Prophet Model (Model P) ---
print("\n--- Training Prophet Model (Model P) ---")

# Use best params found from grid search completed separately.
best_cps = 0.05
best_sm = 'additive'
best_sps = 10.0
best_hps = 1.0

prophet_model = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=False, # Data is monthly
    changepoint_prior_scale=best_cps,
    seasonality_mode=best_sm,
    seasonality_prior_scale=best_sps,
    holidays_prior_scale=best_hps,
    holidays=holiday_events
)
prophet_model.add_regressor('avg_package_count')
prophet_model.add_seasonality(name='monthly', period=30.5, fourier_order=5) # Custom monthly seasonality

print(f"Training Prophet model on data up to: {train_monthly_prophet['ds'].max().date()}")
prophet_model.fit(train_monthly_prophet[['ds', 'y', 'avg_package_count']])
print("Prophet model training complete.")


## 7. Prophet Model (Model P) – Generating Full Period Forecast

This block defines the **forecast horizon** and creates a template `DataFrame` (`future_df_template`) that includes both **historical dates** and **future dates** for prediction.

- The common **evaluation period** starts from `cutoff_date` and ends on `eval_end_common`.
- The `make_future_dataframe` method is used to generate the future dates.
- The `avg_package_count` regressor values are **merged** into this template.
  - Any missing future values are **forward-filled** and **backward-filled**.
- Actual `y` values are also merged into the template for later evaluation.
- Prophet’s `predict` method is called on this template to generate forecasts (`yhat`) for the entire period.
- These predictions are stored in:  
  `future_df_template['prophet_pred_log']`.


In [ ]:
# Define forecast horizon for all models
eval_start_common = cutoff_date # Start of common evaluation period
eval_end_common = pd.to_datetime('2025-02-28') # Final evaluation end date

# Calculate periods needed for make_future_dataframe based on Prophet's training end
periods_from_train_end = (eval_end_common.to_period('M') - train_monthly_prophet['ds'].max().to_period('M')).n
min_forecast_periods = 6 # Ensure a minimum forecast length
periods_for_future_df = max(periods_from_train_end, min_forecast_periods)

# Create a future dataframe template that will be used by all models for consistency
future_df_template = prophet_model.make_future_dataframe(periods=periods_for_future_df, freq='M')
# Merge actual 'y' and 'avg_package_count' for the entire range (historical and future placeholders)
future_df_template = pd.merge(future_df_template, monthly_df[['ds', 'avg_package_count', 'y']], on='ds', how='left')
future_df_template['avg_package_count'] = future_df_template['avg_package_count'].ffill().bfill() # Fill regressor for future dates

# Store Prophet's full forecast in the template
forecast_prophet_full = prophet_model.predict(future_df_template[['ds', 'avg_package_count']]) # Predict on the template
future_df_template['prophet_pred_log'] = forecast_prophet_full['yhat']

print("\nFuture DataFrame Template Head with Prophet Predictions:")
print(future_df_template.head())
print("\nFuture DataFrame Template Tail with Prophet Predictions:")
print(future_df_template.tail())

## 8. Standalone LSTM Model (Model SL) – Data Preparation
This section prepares data for the **Standalone LSTM model (Model SL)**, which directly predicts the **log-transformed submission volume `y`**.

- A **lookback period** is defined, determining how many past time steps are used to predict the next.
- The target variable `y` is **scaled** using `MinMaxScaler` to a range of `[0, 1]`, which is generally beneficial for LSTM performance.
- **Input sequences (`X`)** and corresponding **target values (`y`)** are created:
  - `X` consists of `lookback` previous scaled `y` values.
  - `y` is the next scaled `y` value.
- The **dates (`ds`)** corresponding to the target `y` values are stored for later **alignment and residual calculation**.
- `X` is reshaped to the required **3D format** for LSTM input: `[samples, timesteps, features]`.
- The data is split into:
  - **Training sequences** (up to `cutoff_date`)
  - Sequences for **later prediction**


In [ ]:
# --- Standalone LSTM Model (Model SL - Directly predicting 'y') ---
print("\n--- Preparing Data for Standalone LSTM Model (Model SL) ---")
lookback = 3 # Number of previous months to use for predicting the next month

# Prepare data for standalone LSTM: target is 'y' (log-transformed submission volume)
data_for_standalone_lstm = monthly_df[['ds', 'y']].copy() # Keep 'ds' for easier indexing and merging later

# Scale the target variable 'y'
scaler_standalone_lstm_y = MinMaxScaler(feature_range=(0, 1))
data_for_standalone_lstm['y_scaled'] = scaler_standalone_lstm_y.fit_transform(data_for_standalone_lstm[['y']])

# Create sequences for X (input) and y (target)
X_standalone_sequences, y_standalone_sequences_scaled = [], []
ds_for_y_standalone = [] # To keep track of dates for y_standalone_sequences_scaled (targets)

for i in range(lookback, len(data_for_standalone_lstm)):
    X_standalone_sequences.append(data_for_standalone_lstm['y_scaled'].iloc[i-lookback:i].values)
    y_standalone_sequences_scaled.append(data_for_standalone_lstm['y_scaled'].iloc[i])
    ds_for_y_standalone.append(data_for_standalone_lstm['ds'].iloc[i]) # Date of the target y

X_standalone_sequences = np.array(X_standalone_sequences)
y_standalone_sequences_scaled = np.array(y_standalone_sequences_scaled)
ds_for_y_standalone = pd.Series(ds_for_y_standalone, name='ds') # Convert to Series for easier indexing

# Reshape X for LSTM input: [samples, timesteps, features]
# Here, features = 1 (only using lagged 'y')
X_standalone_sequences = X_standalone_sequences.reshape((X_standalone_sequences.shape[0], X_standalone_sequences.shape[1], 1))

# Training data for Standalone LSTM ends before cutoff_date
# ds_for_y_standalone contains the dates for y_standalone_sequences_scaled
train_indices_standalone = ds_for_y_standalone[ds_for_y_standalone < cutoff_date].index

X_train_standalone_lstm = X_standalone_sequences[train_indices_standalone]
y_train_standalone_lstm_scaled = y_standalone_sequences_scaled[train_indices_standalone]

# Store corresponding 'ds' and actual 'y' (log-transformed) for the training period of Standalone LSTM
# This is needed for calculating residuals of the Standalone LSTM later
ds_train_standalone_lstm = ds_for_y_standalone[train_indices_standalone]
actual_y_train_standalone_lstm_log = data_for_standalone_lstm.set_index('ds').loc[ds_train_standalone_lstm, 'y'].values

print(f"Standalone LSTM training data shapes: X={X_train_standalone_lstm.shape}, y={y_train_standalone_lstm_scaled.shape}")
print(f"Number of training sequences for Standalone LSTM: {len(ds_train_standalone_lstm)}")


## 9. Standalone LSTM Model (Model SL) – Definition and Training

This block defines and trains the **Standalone LSTM model (Model SL)**.

- A **sequential Keras model** is created.
- It consists of:
  - An **LSTM layer** with 32 units, taking input of shape `(lookback, 1)`  
    (i.e., 1 feature: the lagged, scaled `y`)
  - A **Dense output layer** with 1 unit to predict the next scaled `y` value.
- The model is compiled with:
  - `'mse'` (mean squared error) loss
  - `'adam'` optimizer
- The model is trained for **100 epochs** with a **batch size of 4**.
- `EarlyStopping` is used to:
  - Monitor the training loss
  - Prevent overfitting
  - Automatically restore the **best weights**


In [ ]:
# Define and train the standalone LSTM model (using user-specified "best" params)
model_standalone_lstm = Sequential()
lookback = 3 # Number of lagged observations to use (explicitly set as per user update)
model_standalone_lstm.add(LSTM(32, input_shape=(lookback, 1))) # 1 feature: lagged 'y'
model_standalone_lstm.add(Dense(1)) # Output layer for predicting the single next value
model_standalone_lstm.compile(loss='mse', optimizer='adam')

print("Training Standalone LSTM model (Model SL)...")
# Using EarlyStopping to prevent overfitting and restore best weights
standalone_lstm_early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True, verbose=0)
model_standalone_lstm.fit(X_train_standalone_lstm, y_train_standalone_lstm_scaled,
                          epochs=100, 
                          batch_size=4, 
                          verbose=0, # Set to 1 or 2 to see training progress
                          callbacks=[standalone_lstm_early_stopping])
print("Standalone LSTM model training complete.")



## 10. Standalone LSTM Model (Model SL) – Generating Full Period Forecast

After training, the **Standalone LSTM model** is used to generate predictions for the entire **forecast horizon** defined in `future_df_template`.

- Predictions begin from the **end of the LSTM’s training data**.
- An **iterative (auto-regressive)** forecasting process is used:
  - The **last known sequence** from the training data is used as the initial input.
  - The model predicts **one step ahead**.
  - This prediction is then used to **update the input sequence** for the next prediction.
- All **scaled predictions** are collected.
- These predictions are then **inverse-transformed** using `scaler_standalone_lstm_y` to return them to the **original log-transformed scale**.
- The predictions are **aligned with the dates** in `future_df_template` and stored in the `standalone_lstm_pred_log` column.
- Any remaining `NaN`s (e.g., at the beginning if the lookback period isn't fully covered by `ds_for_y_standalone`, or at the end if predictions don’t perfectly align with `future_df_template` length) are **forward-filled**.


In [ ]:
# Generate Standalone LSTM predictions for the full future_df_template horizon
print("Generating full period predictions with Standalone LSTM (Model SL)...")

# Start with the scaled y-values from the training period of the standalone LSTM
# These are the actual scaled values up to the point where training ended.
all_standalone_lstm_preds_scaled = list(y_standalone_sequences_scaled[train_indices_standalone])


# The first input sequence for iterative forecasting is the last sequence from X_train_standalone_lstm
current_input_sequence = X_train_standalone_lstm[-1].reshape(1, lookback, 1)

# Determine how many future steps we need to predict beyond the training data
# to fill up to the end of future_df_template
last_train_date_sl = ds_train_standalone_lstm.max()
num_future_preds_needed = len(future_df_template[future_df_template['ds'] > last_train_date_sl])

for _ in range(num_future_preds_needed):
    pred_scaled = model_standalone_lstm.predict(current_input_sequence, verbose=0)[0,0]
    all_standalone_lstm_preds_scaled.append(pred_scaled)
    # Update sequence: remove oldest, add new prediction
    new_element_for_sequence = np.array([[[pred_scaled]]]) # Shape (1,1,1)
    current_input_sequence = np.append(current_input_sequence[:, 1:, :], new_element_for_sequence, axis=1)

# Inverse transform all predictions (training part + forecasted part)
all_standalone_lstm_preds_log = scaler_standalone_lstm_y.inverse_transform(np.array(all_standalone_lstm_preds_scaled).reshape(-1,1)).flatten()

# Align these predictions with future_df_template
# The predictions correspond to `ds_train_standalone_lstm` and then the future dates
# Create a temporary DataFrame for merging
pred_dates_for_sl_merge = list(ds_train_standalone_lstm) + \
                          list(future_df_template[future_df_template['ds'] > last_train_date_sl]['ds'])

# Ensure lengths match if there are slight discrepancies
max_len = min(len(pred_dates_for_sl_merge), len(all_standalone_lstm_preds_log))
temp_standalone_lstm_preds_df = pd.DataFrame({
    'ds': pred_dates_for_sl_merge[:max_len], 
    'standalone_lstm_pred_log': all_standalone_lstm_preds_log[:max_len]
})

# Merge into future_df_template
future_df_template = pd.merge(future_df_template.drop(columns=['standalone_lstm_pred_log'], errors='ignore'), 
                              temp_standalone_lstm_preds_df, on='ds', how='left')

# Fill any NaNs that might occur due to merge or slight misalignments
# Forward fill is generally appropriate for forecasts
future_df_template['standalone_lstm_pred_log'] = future_df_template['standalone_lstm_pred_log'].ffill()
# If there are still NaNs at the beginning (before LSTM predictions start), fill with a known value or handle as needed
# For this setup, it's assumed LSTM predictions will cover the relevant forecast period.

print("Standalone LSTM full period predictions merged into future_df_template.")
print("Future DataFrame Template Head with SL Predictions:")
print(future_df_template[['ds', 'prophet_pred_log', 'standalone_lstm_pred_log']].head())
print("Future DataFrame Template Tail with SL Predictions:")
print(future_df_template[['ds', 'prophet_pred_log', 'standalone_lstm_pred_log']].tail())


## 11. Residual-Correcting LSTM (for Prophet's Residuals – Model P_RL) – Data Preparation & Training

**Explanation:**  
This section introduces the first **hybrid approach**: using an **LSTM to model the residuals** of the main Prophet model (Model P).  
This hybrid model is referred to as **Model P_RL**.

### Steps Involved:

- **Residual Calculation**:  
  - Prophet’s predictions (`yhat`) on its training data (`train_monthly_prophet`) are obtained.
  - Residuals are calculated as:  
    `actual_y - prophet_yhat`

- **Data Scaling**:  
  - Residuals are scaled using `StandardScaler` (since they are centered around zero).
  - The `avg_package_count` regressor (for this LSTM’s training period) is scaled using `MinMaxScaler`.

- **Sequence Creation**:  
  - Input sequences (`X`) for the LSTM consist of:
    - Lagged **scaled residuals**
    - Lagged **scaled `avg_package_count`**
  - The target (`y`) is the **next scaled residual**.

- **Train/Validation Split**:  
  - The sequenced data is split into **training** and **validation** sets for the `P_RL` LSTM.

- **P_RL LSTM Definition & Training**:
  - A **simplified LSTM model** (`model_p_rl`) is defined:
    - 1 LSTM layer with **16 units**
    - A `Dropout` layer
    - A `Dense` output layer
  - The model is compiled and trained to predict the **scaled residuals**.
  - `EarlyStopping` is used, monitoring **validation loss**.
  
- The model’s performance on the **validation set** is **evaluated and printed**.


In [ ]:
# --- Residual-Correcting LSTM (for Prophet's residuals - Model P_RL) ---
print("\n--- Training Residual-Correcting LSTM (for Prophet's residuals - Model P_RL) ---")

# Get Prophet's predictions on its training data to calculate residuals
prophet_train_preds_df = prophet_model.predict(train_monthly_prophet[['ds', 'avg_package_count']])
p_rl_train_source = pd.merge(train_monthly_prophet[['ds', 'y', 'avg_package_count']],
                             prophet_train_preds_df[['ds', 'yhat']], on='ds', how='inner')
p_rl_train_source['residual'] = p_rl_train_source['y'] - p_rl_train_source['yhat'] # Prophet's residuals

# Scale residuals and the regressor for this LSTM
scaler_p_rl_residual = StandardScaler() # StandardScaler is often good for residuals
p_rl_train_source['residual_scaled'] = scaler_p_rl_residual.fit_transform(p_rl_train_source[['residual']])

scaler_p_rl_pkg_count = MinMaxScaler() # Scaler for avg_package_count for this P_RL LSTM
p_rl_train_source['avg_package_count_scaled'] = scaler_p_rl_pkg_count.fit_transform(p_rl_train_source[['avg_package_count']])

# Create sequences for P_RL LSTM
X_p_rl, y_p_rl_scaled = [], []
ds_for_y_p_rl = [] # Dates for the target residuals

for i in range(lookback, len(p_rl_train_source)):
    # Features: lagged scaled residuals and lagged scaled avg_package_count
    sequence = p_rl_train_source[['residual_scaled', 'avg_package_count_scaled']].iloc[i-lookback:i].values
    X_p_rl.append(sequence)
    y_p_rl_scaled.append(p_rl_train_source['residual_scaled'].iloc[i])
    ds_for_y_p_rl.append(p_rl_train_source['ds'].iloc[i])

X_p_rl, y_p_rl_scaled = np.array(X_p_rl), np.array(y_p_rl_scaled)
ds_for_y_p_rl = pd.Series(ds_for_y_p_rl, name='ds')

# Reshape X_p_rl for LSTM: [samples, timesteps, features]
# Here, features = 2 (lagged scaled residual, lagged scaled avg_package_count)
X_p_rl = X_p_rl.reshape((X_p_rl.shape[0], X_p_rl.shape[1], 2))

# Train/Validation split for P_RL LSTM (e.g., last 20% for validation)
# This split is on the sequences derived from Prophet's training data
validation_split_p_rl = int(len(X_p_rl) * 0.8)
X_train_p_rl, X_val_p_rl = X_p_rl[:validation_split_p_rl], X_p_rl[validation_split_p_rl:]
y_train_p_rl_scaled, y_val_p_rl_scaled = y_p_rl_scaled[:validation_split_p_rl], y_p_rl_scaled[validation_split_p_rl:]
ds_val_p_rl = ds_for_y_p_rl[validation_split_p_rl:] # Dates for the validation set

print(f"P_RL LSTM training data shapes: X={X_train_p_rl.shape}, y={y_train_p_rl_scaled.shape}")
print(f"P_RL LSTM validation data shapes: X={X_val_p_rl.shape}, y={y_val_p_rl_scaled.shape}")

# Define and train the P_RL LSTM model
model_p_rl = Sequential()
model_p_rl.add(LSTM(16, input_shape=(lookback, 2), kernel_regularizer=l2(0.001))) # 2 features
model_p_rl.add(Dropout(0.2))
model_p_rl.add(Dense(1)) # Output layer for predicting the scaled residual
model_p_rl.compile(loss='mse', optimizer='adam')

print("Training P_RL LSTM model (to predict Prophet residuals)...")
p_rl_early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=0)
history_p_rl = model_p_rl.fit(X_train_p_rl, y_train_p_rl_scaled,
                              epochs=150,
                              batch_size=2,
                              validation_data=(X_val_p_rl, y_val_p_rl_scaled),
                              verbose=0,
                              callbacks=[p_rl_early_stopping])

print("P_RL LSTM model training complete.")

# Evaluate P_RL model on its validation set (predicting Prophet's residuals)
val_loss_p_rl = model_p_rl.evaluate(X_val_p_rl, y_val_p_rl_scaled, verbose=0)
print(f"P_RL LSTM Validation MSE (on scaled residuals): {val_loss_p_rl:.4f}")

# Store actual residuals for the P_RL validation period for later analysis if needed
actual_residuals_val_p_rl_scaled = y_val_p_rl_scaled
predicted_residuals_val_p_rl_scaled = model_p_rl.predict(X_val_p_rl, verbose=0).flatten()

actual_residuals_val_p_rl = scaler_p_rl_residual.inverse_transform(actual_residuals_val_p_rl_scaled.reshape(-1, 1)).flatten()
predicted_residuals_val_p_rl = scaler_p_rl_residual.inverse_transform(predicted_residuals_val_p_rl_scaled.reshape(-1, 1)).flatten()

val_mae_p_rl_residuals = mean_absolute_error(actual_residuals_val_p_rl, predicted_residuals_val_p_rl)
print(f"P_RL LSTM Validation MAE (on original scale residuals): {val_mae_p_rl_residuals:.4f}")


## 12. P_RL Model – Generating Full Period Residual Forecasts and Combining with Prophet

**Explanation:**  
This block uses the trained **P_RL LSTM** to forecast **Prophet's residuals** over the entire period covered by `future_df_template`.  
These forecasted residuals are then **added back** to Prophet's main forecast (`prophet_pred_log`) to create the **hybrid P + P_RL prediction**.

---

### 📌 Prepare Input Data for P_RL Forecasting:

- `future_df_template` already contains:
  - Prophet's predictions (`prophet_pred_log`)
  - `avg_package_count`

- For **dates up to `cutoff_date`**:
  - "Actual" `y` values are used to calculate historical **residuals** for the P_RL model's lookback window.

- For **dates beyond `cutoff_date`**:
  - Prophet’s own predictions are used as the base to estimate residuals that **P_RL will try to correct**.

- Both residuals and `avg_package_count` are **scaled** using the same scalers (`scaler_p_rl_residual`, `scaler_p_rl_pkg_count`) fitted during P_RL training.

---

### 🔁 Iterative Residual Forecasting:

- An **iterative (auto-regressive)** approach is used (similar to the Standalone LSTM model):
  - The **P_RL LSTM** predicts one **scaled residual** at a time.
  - The input sequence is updated after each step with:
    - The **new forecasted scaled residual**
    - The corresponding **scaled `avg_package_count`**

---

### 🔄 Inverse Transform and Combine:

- The forecasted **scaled residuals** are **inverse-transformed** back to their original scale.
- These residuals are then **added to Prophet’s log-transformed predictions** (`prophet_pred_log`) to produce the final hybrid prediction.
- The combined output is stored in:  
  `future_df_template['prophet_plus_p_rl_pred_log']`

- Any resulting `NaN`s from the lookback period at the beginning are **forward-filled**.


In [ ]:
# --- P_RL Model - Generating Full Period Residual Forecasts and Combining ---
print("\n--- Generating Full Period P_RL Residual Forecasts & Combining with Prophet ---")

# We need to create the input sequences for P_RL for the entire future_df_template period.
# This involves:
# 1. Prophet's predictions ('prophet_pred_log') from future_df_template.
# 2. 'avg_package_count' from future_df_template.
# 3. For historical periods (before cutoff_date), use actual 'y' to calculate residuals.
#    For future periods (after cutoff_date), the "residual" is effectively 0 if we assume Prophet's forecast is the base,
#    or we can try to predict deviations from Prophet's future forecasts.
#    Here, we'll predict corrections to Prophet's 'yhat' for the whole future_df_template.

# Create a temporary DataFrame for P_RL inputs, aligned with future_df_template
p_rl_input_df = future_df_template[['ds', 'y', 'prophet_pred_log', 'avg_package_count']].copy()

# Calculate residuals: actual 'y' - prophet_pred_log where 'y' is available,
# otherwise, the residual to predict is against prophet_pred_log (so, if Prophet is perfect, residual is 0)
# For the P_RL model, it was trained on residuals = y_actual - prophet_yhat_on_train_data.
# When forecasting, we want to predict future residuals: future_y_actual - future_prophet_yhat.
# Since future_y_actual is unknown, P_RL predicts the expected error of Prophet.

# For the initial lookback period, we need historical residuals.
# We use the residuals calculated on Prophet's training data (p_rl_train_source)
# and then transition to iterative prediction.

# Scale avg_package_count for the entire period using the P_RL scaler
p_rl_input_df['avg_package_count_scaled'] = scaler_p_rl_pkg_count.transform(p_rl_input_df[['avg_package_count']])

# Initialize a column for scaled residuals in p_rl_input_df
p_rl_input_df['residual_scaled_for_pred'] = np.nan

# Fill known scaled residuals from the P_RL training phase
# Align p_rl_train_source (which has 'residual_scaled') with p_rl_input_df
temp_merge_df = pd.merge(p_rl_input_df[['ds']],
                         p_rl_train_source[['ds', 'residual_scaled']],
                         on='ds',
                         how='left')
p_rl_input_df['residual_scaled_for_pred'] = temp_merge_df['residual_scaled']


# Iteratively predict residuals for dates where 'residual_scaled_for_pred' is NaN
# These NaNs will typically start after the P_RL training data ends.

all_p_rl_residual_preds_scaled = []
# Find the first index where iterative prediction should start
# This is typically lookback steps after the start of p_rl_input_df, or after known residuals end.
first_pred_idx = lookback 

# If there are pre-filled residuals from training, start predicting after them.
last_known_residual_idx = p_rl_input_df[p_rl_input_df['residual_scaled_for_pred'].notna()].index.max()
if pd.notna(last_known_residual_idx) and last_known_residual_idx + 1 > first_pred_idx :
    first_pred_idx = last_known_residual_idx + 1
    # Populate all_p_rl_residual_preds_scaled with known historical scaled residuals up to this point
    all_p_rl_residual_preds_scaled.extend(p_rl_input_df['residual_scaled_for_pred'].iloc[:first_pred_idx].tolist())
else: # If no known residuals or they end too early, fill initial part with zeros or a mean
    # For simplicity, if we start predicting from 'lookback', the first few values in all_p_rl_residual_preds_scaled
    # will be NaNs if not filled from p_rl_train_source.
    # Let's ensure the list starts with values that can form the first input sequence.
    # We will use the actual known residuals from p_rl_train_source for the initial part of all_p_rl_residual_preds_scaled.
    # The loop below will then append new predictions.
    
    # If first_pred_idx is still 'lookback', it means we don't have enough history from p_rl_train_source
    # directly in p_rl_input_df to start iterative prediction immediately.
    # This logic assumes p_rl_train_source's residuals are the ground truth for the start.
    # We copy them over.
    
    # The critical part is the input sequence for the *first prediction*.
    # This sequence must come from data scaled consistently with P_RL's training.
    
    # Let's reconstruct the scaled values for the initial sequence from p_rl_input_df
    # using data up to the point before predictions begin.
    
    # The `all_p_rl_residual_preds_scaled` list will store all scaled residuals: known historical + newly predicted.
    # It should be populated with all available historical scaled residuals first.
    
    # Correctly initialize `all_p_rl_residual_preds_scaled` with values from `p_rl_input_df['residual_scaled_for_pred']`
    # up to `first_pred_idx -1`. The loop will then predict from `first_pred_idx` onwards.
    
    # The issue is that `p_rl_input_df['residual_scaled_for_pred']` might have NaNs even before `first_pred_idx`
    # if `p_rl_train_source` didn't cover the beginning of `future_df_template`.
    # For simplicity, let's assume `p_rl_train_source` provides enough history.
    # The `temp_merge_df` step should have populated these.
    
    # If `all_p_rl_residual_preds_scaled` is shorter than `first_pred_idx`, it means we have a gap.
    # This typically happens if `p_rl_train_source` doesn't align with the start of `future_df_template`.
    # We will fill initial NaNs in `residual_scaled_for_pred` with 0 for simplicity before iterative loop.
    p_rl_input_df['residual_scaled_for_pred'] = p_rl_input_df['residual_scaled_for_pred'].fillna(0) # Risky, but for now.
    all_p_rl_residual_preds_scaled = p_rl_input_df['residual_scaled_for_pred'].iloc[:first_pred_idx].tolist()


print(f"Starting P_RL iterative prediction from index: {first_pred_idx} (date: {p_rl_input_df['ds'].iloc[first_pred_idx]})")

for i in range(first_pred_idx, len(p_rl_input_df)):
    # Get the last 'lookback' scaled residuals and avg_package_count_scaled
    # Residuals for the sequence come from `all_p_rl_residual_preds_scaled` which is being built
    current_residuals_scaled_sequence = np.array(all_p_rl_residual_preds_scaled[i-lookback:i])
    current_pkg_count_scaled_sequence = p_rl_input_df['avg_package_count_scaled'].iloc[i-lookback:i].values
    
    # Combine them into the 2-feature input for P_RL LSTM
    current_input_sequence_p_rl = np.stack((current_residuals_scaled_sequence, current_pkg_count_scaled_sequence), axis=-1)
    current_input_sequence_p_rl = current_input_sequence_p_rl.reshape(1, lookback, 2) # Reshape for LSTM
    
    # Predict the next scaled residual
    pred_scaled_residual = model_p_rl.predict(current_input_sequence_p_rl, verbose=0)[0,0]
    all_p_rl_residual_preds_scaled.append(pred_scaled_residual)

# Ensure `all_p_rl_residual_preds_scaled` has the same length as `p_rl_input_df`
# If it's shorter (e.g. due to loop range), pad with last prediction or zero
if len(all_p_rl_residual_preds_scaled) < len(p_rl_input_df):
    padding = [all_p_rl_residual_preds_scaled[-1]] * (len(p_rl_input_df) - len(all_p_rl_residual_preds_scaled))
    all_p_rl_residual_preds_scaled.extend(padding)
elif len(all_p_rl_residual_preds_scaled) > len(p_rl_input_df):
    all_p_rl_residual_preds_scaled = all_p_rl_residual_preds_scaled[:len(p_rl_input_df)]


# Inverse transform the predicted scaled residuals
predicted_residuals_p_rl_log_scale = scaler_p_rl_residual.inverse_transform(np.array(all_p_rl_residual_preds_scaled).reshape(-1,1)).flatten()

# Add these predicted residuals to Prophet's main forecast
future_df_template['p_rl_predicted_residual_log'] = predicted_residuals_p_rl_log_scale
future_df_template['prophet_plus_p_rl_pred_log'] = future_df_template['prophet_pred_log'] + future_df_template['p_rl_predicted_residual_log']

# Handle potential NaNs from lookback at the beginning if any (e.g. if iterative prediction didn't cover all)
# The `all_p_rl_residual_preds_scaled` should cover the whole range.
# NaNs in 'prophet_plus_p_rl_pred_log' would primarily come from NaNs in 'prophet_pred_log' or 'p_rl_predicted_residual_log'.
# `future_df_template['prophet_pred_log']` should be full from Prophet.
# `p_rl_predicted_residual_log` should be full from the iterative loop.
# A ffill can handle any edge cases.
future_df_template['prophet_plus_p_rl_pred_log'] = future_df_template['prophet_plus_p_rl_pred_log'].ffill()


print("\nP_RL residual forecasts generated and combined with Prophet.")
print("Future DataFrame Template Head with P+P_RL Predictions:")
print(future_df_template[['ds', 'prophet_pred_log', 'p_rl_predicted_residual_log', 'prophet_plus_p_rl_pred_log']].head())
print("\nFuture DataFrame Template Tail with P+P_RL Predictions:")
print(future_df_template[['ds', 'prophet_pred_log', 'p_rl_predicted_residual_log', 'prophet_plus_p_rl_pred_log']].tail())

# Clean up temporary columns if desired
# future_df_template.drop(columns=['p_rl_predicted_residual_log'], inplace=True, errors='ignore')


## 13. Residual-Correcting LSTM (for Standalone LSTM's Residuals – Model SL_RL) – Data Preparation & Training

**Explanation:**  
This section implements the second hybrid approach: an **LSTM that models the residuals of the Standalone LSTM (Model SL)**.  
This is referred to as **Model SL_RL**. The methodology closely mirrors that of Model P_RL, but instead works with **Model SL's outputs and residuals**.

---

### 🔍 Residual Calculation:

- Predictions from the trained **Standalone LSTM (`model_standalone_lstm`)** are made on its **training data (`X_train_standalone_lstm`)**.
- These are **scaled predictions**, which are then **inverse-transformed** to recover the **log scale** (`predicted_y_train_sl_log`).
- Residuals are calculated as:  
  `actual_y_train_standalone_lstm_log - predicted_y_train_sl_log`

---

### ⚖️ Data Scaling:

- The residuals are **scaled** using `StandardScaler` to center them around zero and normalize the variance.

---

### 📐 Sequence Creation:

- Input sequences (`X`) are constructed from **lagged, scaled residuals** of Model SL.
- Target values (`y`) are the **next scaled residual** in the sequence.
- No external regressor (e.g., `avg_package_count`) is used in this model, under the assumption that **Model SL's residuals are primarily autocorrelated** and self-driven.

---

### ✂️ Train/Validation Split:

- The residual sequences are split into **training** and **validation** sets, typically using an 80/20 ratio.

---

### 🧠 SL_RL LSTM Definition & Training:

- A simple **LSTM model (`model_sl_rl`)** is defined:
  - One LSTM layer with **16 units**
  - A **Dropout** layer to prevent overfitting
  - A **Dense output** layer for residual prediction

- The model is compiled with:
  - **Loss function**: `'mse'` (Mean Squared Error)
  - **Optimizer**: `'adam'`

- The model is trained using:
  - A **small batch size** (e.g., 2)
  - Up to **150 epochs**
  - **EarlyStopping** based on validation loss with best weight restoration

- The model’s performance is evaluated on the **validation set**, and the **Validation MSE** is reported.


In [ ]:
# --- Residual-Correcting LSTM (for Standalone LSTM's residuals - Model SL_RL) ---
print("\n--- Training Residual-Correcting LSTM (for Standalone LSTM's residuals - Model SL_RL) ---")

# Get Standalone LSTM's predictions on its training data to calculate residuals
# These predictions were originally scaled, so inverse transform them first.
predicted_y_train_sl_scaled = model_standalone_lstm.predict(X_train_standalone_lstm, verbose=0)
predicted_y_train_sl_log = scaler_standalone_lstm_y.inverse_transform(predicted_y_train_sl_scaled).flatten()

# actual_y_train_standalone_lstm_log was stored during SL data preparation (Section 8)
sl_rl_train_source = pd.DataFrame({
    'ds': ds_train_standalone_lstm, # Dates corresponding to SL training targets
    'actual_y_log': actual_y_train_standalone_lstm_log,
    'predicted_y_sl_log': predicted_y_train_sl_log
})
sl_rl_train_source['residual_sl'] = sl_rl_train_source['actual_y_log'] - sl_rl_train_source['predicted_y_sl_log']

# Scale residuals for SL_RL LSTM
scaler_sl_rl_residual = StandardScaler()
sl_rl_train_source['residual_sl_scaled'] = scaler_sl_rl_residual.fit_transform(sl_rl_train_source[['residual_sl']])

# Create sequences for SL_RL LSTM (only using lagged scaled residuals of SL)
X_sl_rl, y_sl_rl_scaled = [], []
ds_for_y_sl_rl = []

for i in range(lookback, len(sl_rl_train_source)):
    # Feature: lagged scaled residual from Standalone LSTM
    sequence = sl_rl_train_source['residual_sl_scaled'].iloc[i-lookback:i].values.reshape(-1, 1) # Reshape for 1 feature
    X_sl_rl.append(sequence)
    y_sl_rl_scaled.append(sl_rl_train_source['residual_sl_scaled'].iloc[i])
    ds_for_y_sl_rl.append(sl_rl_train_source['ds'].iloc[i])

X_sl_rl, y_sl_rl_scaled = np.array(X_sl_rl), np.array(y_sl_rl_scaled)
ds_for_y_sl_rl = pd.Series(ds_for_y_sl_rl, name='ds')

# Reshape X_sl_rl for LSTM: [samples, timesteps, features=1]
X_sl_rl = X_sl_rl.reshape((X_sl_rl.shape[0], X_sl_rl.shape[1], 1))

# Train/Validation split for SL_RL LSTM
validation_split_sl_rl = int(len(X_sl_rl) * 0.8)
X_train_sl_rl, X_val_sl_rl = X_sl_rl[:validation_split_sl_rl], X_sl_rl[validation_split_sl_rl:]
y_train_sl_rl_scaled, y_val_sl_rl_scaled = y_sl_rl_scaled[:validation_split_sl_rl], y_sl_rl_scaled[validation_split_sl_rl:]

print(f"SL_RL LSTM training data shapes: X={X_train_sl_rl.shape}, y={y_train_sl_rl_scaled.shape}")
print(f"SL_RL LSTM validation data shapes: X={X_val_sl_rl.shape}, y={y_val_sl_rl_scaled.shape}")

# Define and train the SL_RL LSTM model
model_sl_rl = Sequential()
model_sl_rl.add(LSTM(16, input_shape=(lookback, 1), kernel_regularizer=l2(0.001))) # 1 feature: SL residual
model_sl_rl.add(Dropout(0.2))
model_sl_rl.add(Dense(1)) # Output layer for predicting the scaled SL residual
model_sl_rl.compile(loss='mse', optimizer='adam')

print("Training SL_RL LSTM model (to predict Standalone LSTM residuals)...")
sl_rl_early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=0)
history_sl_rl = model_sl_rl.fit(X_train_sl_rl, y_train_sl_rl_scaled,
                              epochs=150,
                              batch_size=2,
                              validation_data=(X_val_sl_rl, y_val_sl_rl_scaled),
                              verbose=0,
                              callbacks=[sl_rl_early_stopping])
print("SL_RL LSTM model training complete.")

val_loss_sl_rl = model_sl_rl.evaluate(X_val_sl_rl, y_val_sl_rl_scaled, verbose=0)
print(f"SL_RL LSTM Validation MSE (on scaled SL residuals): {val_loss_sl_rl:.4f}")

## 14. SL_RL Model – Generating Full Period Residual Forecasts and Combining with Standalone LSTM

**Explanation:**  
This block uses the trained **SL_RL LSTM** to forecast the **Standalone LSTM (Model SL)** residuals over the full forecast horizon defined in `future_df_template`.  
The predicted residuals are added to Model SL’s main predictions (`standalone_lstm_pred_log`) to generate the **hybrid SL + SL_RL forecast**.

---

### 🛠 Prepare Input Data for SL_RL Forecasting:

- `future_df_template` already contains:
  - Predictions from Model SL (`standalone_lstm_pred_log`)

- For dates **up to `cutoff_date`**:
  - "Actual" `y` values are available and are used to calculate historical **residuals** for the SL_RL model’s lookback window.

- For dates **after `cutoff_date`**:
  - Model SL’s predictions are used as the base.
  - SL_RL attempts to correct their residual errors.

- These residuals are **scaled** using `scaler_sl_rl_residual`, which was fit during SL_RL training.

---

### 🔁 Iterative Residual Forecasting:

- The **SL_RL LSTM** predicts one **scaled residual** at a time.
- After each prediction:
  - The result is used to **update the input sequence**.
  - The process continues until the end of the forecast horizon.

---

### 🔄 Inverse Transform and Combine:

- Forecasted residuals (in scaled form) are **inverse-transformed** to their original scale.
- These are then **added to the log-transformed predictions** from Model SL (`standalone_lstm_pred_log`).
- The combined hybrid output is stored in:  
  `future_df_template['standalone_lstm_plus_sl_rl_pred_log']`


In [ ]:
# --- SL_RL Model - Generating Full Period Residual Forecasts & Combining ---
print("\n--- Generating Full Period SL_RL Residual Forecasts & Combining with Standalone LSTM ---")

# We need input sequences for SL_RL for the entire future_df_template period.
# This uses 'standalone_lstm_pred_log' as the base.
sl_rl_input_df = future_df_template[['ds', 'y', 'standalone_lstm_pred_log']].copy()
sl_rl_input_df.rename(columns={'standalone_lstm_pred_log': 'base_pred_log'}, inplace=True)

# Calculate residuals: actual 'y' - base_pred_log where 'y' is available.
# For the training part of SL_RL, we use the residuals from sl_rl_train_source.
# For the forecasting part, SL_RL predicts corrections to 'base_pred_log'.
sl_rl_input_df['residual_for_pred'] = sl_rl_input_df['y'] - sl_rl_input_df['base_pred_log']

# Scale these residuals using the SL_RL scaler.
# Note: transform only, as fit was done on training residuals.
# We need to handle NaNs if 'y' is not available for the full future_df_template range.
# For forecasting, the "residual" is what SL_RL predicts as deviation from standalone_lstm_pred_log.

# Initialize a column for scaled residuals in sl_rl_input_df
sl_rl_input_df['residual_sl_scaled_for_pred'] = np.nan

# Fill known scaled residuals from the SL_RL training phase
temp_merge_sl_rl_df = pd.merge(sl_rl_input_df[['ds']],
                               sl_rl_train_source[['ds', 'residual_sl_scaled']],
                               on='ds',
                               how='left')
sl_rl_input_df['residual_sl_scaled_for_pred'] = temp_merge_sl_rl_df['residual_sl_scaled']

# Iteratively predict SL residuals
all_sl_rl_residual_preds_scaled = []
first_pred_idx_sl_rl = lookback

last_known_sl_rl_residual_idx = sl_rl_input_df[sl_rl_input_df['residual_sl_scaled_for_pred'].notna()].index.max()

if pd.notna(last_known_sl_rl_residual_idx) and last_known_sl_rl_residual_idx + 1 > first_pred_idx_sl_rl:
    first_pred_idx_sl_rl = last_known_sl_rl_residual_idx + 1
    all_sl_rl_residual_preds_scaled.extend(sl_rl_input_df['residual_sl_scaled_for_pred'].iloc[:first_pred_idx_sl_rl].tolist())
else:
    # Fill initial part with zeros if no history or not enough. This assumes residuals start around zero.
    sl_rl_input_df['residual_sl_scaled_for_pred'] = sl_rl_input_df['residual_sl_scaled_for_pred'].fillna(0)
    all_sl_rl_residual_preds_scaled = sl_rl_input_df['residual_sl_scaled_for_pred'].iloc[:first_pred_idx_sl_rl].tolist()

print(f"Starting SL_RL iterative prediction from index: {first_pred_idx_sl_rl} (date: {sl_rl_input_df['ds'].iloc[first_pred_idx_sl_rl]})")

for i in range(first_pred_idx_sl_rl, len(sl_rl_input_df)):
    current_residuals_sl_scaled_sequence = np.array(all_sl_rl_residual_preds_scaled[i-lookback:i])
    current_input_sequence_sl_rl = current_residuals_sl_scaled_sequence.reshape(1, lookback, 1) # 1 feature
    
    pred_scaled_sl_residual = model_sl_rl.predict(current_input_sequence_sl_rl, verbose=0)[0,0]
    all_sl_rl_residual_preds_scaled.append(pred_scaled_sl_residual)

# Ensure lengths match
if len(all_sl_rl_residual_preds_scaled) < len(sl_rl_input_df):
    padding_sl_rl = [all_sl_rl_residual_preds_scaled[-1]] * (len(sl_rl_input_df) - len(all_sl_rl_residual_preds_scaled))
    all_sl_rl_residual_preds_scaled.extend(padding_sl_rl)
elif len(all_sl_rl_residual_preds_scaled) > len(sl_rl_input_df):
    all_sl_rl_residual_preds_scaled = all_sl_rl_residual_preds_scaled[:len(sl_rl_input_df)]

# Inverse transform the predicted scaled SL residuals
predicted_residuals_sl_rl_log_scale = scaler_sl_rl_residual.inverse_transform(np.array(all_sl_rl_residual_preds_scaled).reshape(-1,1)).flatten()

# Add these predicted residuals to Standalone LSTM's main forecast
future_df_template['sl_rl_predicted_residual_log'] = predicted_residuals_sl_rl_log_scale
future_df_template['standalone_lstm_plus_sl_rl_pred_log'] = future_df_template['standalone_lstm_pred_log'] + future_df_template['sl_rl_predicted_residual_log']
future_df_template['standalone_lstm_plus_sl_rl_pred_log'] = future_df_template['standalone_lstm_plus_sl_rl_pred_log'].ffill()

print("\nSL_RL residual forecasts generated and combined with Standalone LSTM.")
print("Future DataFrame Template Head with SL+SL_RL Predictions:")
print(future_df_template[['ds', 'standalone_lstm_pred_log', 'sl_rl_predicted_residual_log', 'standalone_lstm_plus_sl_rl_pred_log']].head())
print("\nFuture DataFrame Template Tail with SL+SL_RL Predictions:")
print(future_df_template[['ds', 'standalone_lstm_pred_log', 'sl_rl_predicted_residual_log', 'standalone_lstm_plus_sl_rl_pred_log']].tail())


## 15. Bayesian Model Averaging (BMA)

**Explanation:**  
**Bayesian Model Averaging (BMA)** is a technique for combining forecasts from multiple models.  
Rather than selecting a single "best" model, BMA creates a **weighted average forecast**, where each model’s contribution is proportional to its **reliability or past performance**.

In this demonstration, we’ll use **simple, pre-defined weights** for combining model outputs.  
> *Note: In a production setup, these weights can and should be optimized — for example, using validation-set error metrics like MAE, RMSE, or log-likelihood.*

---

### 📈 Models Included in the Averaging:

- **Prophet (Model P)**  
  Forecast stored in: `prophet_pred_log`

- **Standalone LSTM (Model SL)**  
  Forecast stored in: `standalone_lstm_pred_log`

- **Prophet + P_RL Hybrid Model**  
  Forecast stored in: `prophet_plus_p_rl_pred_log`

- **Standalone LSTM + SL_RL Hybrid Model**  
  Forecast stored in: `standalone_lstm_plus_sl_rl_pred_log`

---

### 🧮 Averaging Logic:

- All model outputs are in the **log-transformed space**.
- BMA produces a final forecast as a **weighted sum of log-scale predictions**.
- Each model can be assigned a weight (e.g., 0.25 each for equal weighting).
- The result reflects the **combined strength** of different modeling strategies — both classical (Prophet) and neural (LSTM), standalone and hybrid.

---

### ℹ️ Note:

- The `avg_package_count` regressor is **implicitly included** in models based on Prophet, and partially in hybrid versions as well.
- BMA doesn’t re-use this regressor directly but benefits from models that already incorporate it during training.


In [ ]:
# --- Bayesian Model Averaging (BMA) ---
print("\n--- Performing Bayesian Model Averaging ---")

# Define weights for each model (these are illustrative and can be optimized)
# For simplicity, let's give equal weight to the two hybrid models,
# and perhaps less to the standalone ones if hybrids are expected to be better.
# Example weights:
weights = {
    'prophet': 0.15,
    'standalone_lstm': 0.15,
    'prophet_plus_p_rl': 0.35,
    'standalone_lstm_plus_sl_rl': 0.35
}

# Ensure weights sum to 1 (or normalize them)
total_weight = sum(weights.values())
if not np.isclose(total_weight, 1.0):
    print(f"Warning: Weights do not sum to 1 (sum={total_weight}). Normalizing weights.")
    weights = {model: wt / total_weight for model, wt in weights.items()}

# Check if all necessary prediction columns exist
required_cols_for_bma = ['prophet_pred_log', 'standalone_lstm_pred_log', 
                         'prophet_plus_p_rl_pred_log', 'standalone_lstm_plus_sl_rl_pred_log']
missing_cols = [col for col in required_cols_for_bma if col not in future_df_template.columns]

if missing_cols:
    raise ValueError(f"Missing columns for BMA in future_df_template: {missing_cols}")

# Fill NaNs in prediction columns with a fallback (e.g., ffill then bfill, or with Prophet's prediction)
# This is crucial as NaNs in any component will result in NaN for BMA.
for col in required_cols_for_bma:
    if future_df_template[col].isnull().any():
        print(f"Warning: NaNs found in {col}. Forward-filling then backward-filling.")
        future_df_template[col] = future_df_template[col].ffill().bfill()
    # As a final fallback, if still NaNs (e.g. entire column was NaN for some reason)
    if future_df_template[col].isnull().any():
         future_df_template[col] = future_df_template['prophet_pred_log'] # Fallback to Prophet
         print(f"Warning: NaNs persisted in {col} after ffill/bfill. Filled with Prophet predictions.")


# Calculate the BMA forecast (weighted average of log-transformed predictions)
future_df_template['bma_pred_log'] = (
    weights['prophet'] * future_df_template['prophet_pred_log'] +
    weights['standalone_lstm'] * future_df_template['standalone_lstm_pred_log'] +
    weights['prophet_plus_p_rl'] * future_df_template['prophet_plus_p_rl_pred_log'] +
    weights['standalone_lstm_plus_sl_rl'] * future_df_template['standalone_lstm_plus_sl_rl_pred_log']
)

print("BMA log predictions calculated.")
print("Future DataFrame Template Head with BMA Predictions:")
print(future_df_template[['ds', 'prophet_pred_log', 'standalone_lstm_pred_log', 'prophet_plus_p_rl_pred_log', 'standalone_lstm_plus_sl_rl_pred_log', 'bma_pred_log']].head())
print("\nFuture DataFrame Template Tail with BMA Predictions:")
print(future_df_template[['ds', 'prophet_pred_log', 'standalone_lstm_pred_log', 'prophet_plus_p_rl_pred_log', 'standalone_lstm_plus_sl_rl_pred_log', 'bma_pred_log']].tail())



## 16. Inverse Transform Predictions

**Explanation:**  
All model predictions are currently in the **log-transformed scale** because of the `np.log1p` transformation applied to the target variable `y` (see Section 3).  
To make these predictions interpretable and comparable to the **original submission volumes**, we need to **inverse transform** them using `np.expm1`.

---

### 🔄 Predictions to Inverse Transform:

- `prophet_pred_log`
- `standalone_lstm_pred_log`
- `prophet_plus_p_rl_pred_log`
- `standalone_lstm_plus_sl_rl_pred_log`
- `bma_pred_log` (Bayesian Model Averaged output)

Each of these columns will be inverse transformed using:

```python
np.expm1(predicted_log_value)


In [ ]:
# --- Inverse Transform Predictions ---
print("\n--- Inverse Transforming Predictions to Original Scale ---")

# Merge original 'y' values for comparison
future_df_template = pd.merge(future_df_template, monthly_df[['ds', 'y_original']], on='ds', how='left')

# List of prediction columns (log scale)
log_pred_columns = [
    'prophet_pred_log',
    'standalone_lstm_pred_log',
    'prophet_plus_p_rl_pred_log',
    'standalone_lstm_plus_sl_rl_pred_log',
    'bma_pred_log'
]

# Inverse transform each prediction column
for log_col in log_pred_columns:
    original_scale_col = log_col.replace('_log', '_orig')
    # Ensure the column exists before trying to transform
    if log_col in future_df_template.columns:
        future_df_template[original_scale_col] = np.expm1(future_df_template[log_col])
        # Handle potential negative predictions if models behave unexpectedly (np.expm1 can handle negatives, but resulting values might be < 0)
        # For submission counts, predictions should ideally be non-negative.
        future_df_template[original_scale_col] = np.maximum(0, future_df_template[original_scale_col]) 
    else:
        print(f"Warning: Log prediction column {log_col} not found for inverse transformation.")

print("Predictions inverse transformed to original scale.")
print("Future DataFrame Template Head with Original Scale Predictions:")
cols_to_show = ['ds', 'y_original'] + [col.replace('_log', '_orig') for col in log_pred_columns if col.replace('_log', '_orig') in future_df_template.columns]
print(future_df_template[cols_to_show].head())
print("\nFuture DataFrame Template Tail with Original Scale Predictions:")
print(future_df_template[cols_to_show].tail())



## 17. Evaluation

**Explanation:**  
In this final step, we evaluate the performance of **all forecasting models** on the **original (non-log-transformed) scale**.  
Two key evaluation metrics are used:

- **Root Mean Squared Error (RMSE)**
- **Mean Absolute Error (MAE)**

---

### 📅 Evaluation Period:

- The evaluation is performed over a **common time range**, defined by:
  - `eval_start_common` → typically equal to `cutoff_date`
  - `eval_end_common` → end of the forecast horizon

---

### 📊 Evaluation Workflow:

1. A new `DataFrame` called `evaluation_df` is created.
   - It includes:
     - `ds` (date column)
     - `y_original` (true values on original scale)
     - All **model predictions** after inverse transformation

2. For each model:
   - RMSE is calculated:
     ```python
     sqrt(mean_squared_error(actual, predicted))
     ```
   - MAE is calculated:
     ```python
     mean_absolute_error(actual, predicted)
     ```

3. The results for each model are stored in a **summary table** (e.g., `metrics_summary_df`) for easy **side-by-side comparison**.

---

### ✅ Metrics Summary Output:
The summary `DataFrame` might look like this:

| Model                    | RMSE    | MAE     |
|-------------------------|---------|---------|
| Prophet (P)


In [ ]:
# --- Evaluation ---
print("\n--- Evaluating Model Performance on Original Scale ---")

# Define the common evaluation period (already defined as eval_start_common and eval_end_common)
evaluation_period_df = future_df_template[
    (future_df_template['ds'] >= eval_start_common) &
    (future_df_template['ds'] <= eval_end_common)
].copy()

# Ensure 'y_original' is present for evaluation
if 'y_original' not in evaluation_period_df.columns or evaluation_period_df['y_original'].isnull().all():
    raise ValueError("'y_original' is missing or all NaNs in the evaluation period. Cannot evaluate.")

# Drop rows where y_original is NaN for fair comparison (if any future dates had no actuals)
evaluation_period_df.dropna(subset=['y_original'], inplace=True)

if evaluation_period_df.empty:
    print("Warning: Evaluation period is empty after dropping NaNs in y_original. No evaluation possible.")
    results_summary = pd.DataFrame(columns=['Model', 'RMSE', 'MAE'])
else:
    models_to_evaluate = {
        'Prophet (P)': 'prophet_pred_orig',
        'Standalone LSTM (SL)': 'standalone_lstm_pred_orig',
        'Prophet + P_RL': 'prophet_plus_p_rl_pred_orig',
        'SL + SL_RL': 'standalone_lstm_plus_sl_rl_pred_orig',
        'BMA': 'bma_pred_orig'
    }

    results = []

    for model_name, pred_col_orig in models_to_evaluate.items():
        if pred_col_orig in evaluation_period_df.columns and not evaluation_period_df[pred_col_orig].isnull().all():
            actuals = evaluation_period_df['y_original']
            predictions = evaluation_period_df[pred_col_orig]
            
            # Ensure no NaNs in predictions for the slice being evaluated
            valid_indices = actuals.notna() & predictions.notna()
            if not valid_indices.any():
                print(f"Warning: No valid (non-NaN) actuals or predictions for {model_name} in evaluation period.")
                rmse = np.nan
                mae = np.nan
            else:
                rmse = np.sqrt(mean_squared_error(actuals[valid_indices], predictions[valid_indices]))
                mae = mean_absolute_error(actuals[valid_indices], predictions[valid_indices])
            
            results.append({'Model': model_name, 'RMSE': rmse, 'MAE': mae})
            print(f"{model_name} - RMSE: {rmse:.4f}, MAE: {mae:.4f}")
        else:
            print(f"Warning: Prediction column {pred_col_orig} for model {model_name} not found or all NaNs in evaluation data. Skipping.")
            results.append({'Model': model_name, 'RMSE': np.nan, 'MAE': np.nan})

    results_summary = pd.DataFrame(results)

print("\n--- Model Evaluation Summary (Original Scale) ---")
print(f"Evaluation Period: {evaluation_period_df['ds'].min().date()} to {evaluation_period_df['ds'].max().date()}")
if results_summary.empty:
    print("No models were evaluated.")
else:
    print(results_summary.sort_values(by='RMSE'))

# Display the final few predictions for context
print("\nFinal Predictions Table (Original Scale - Last 12 Months of Future DF or Full Eval Period):")
display_df_end_limit = min(len(future_df_template), 12) 
if not evaluation_period_df.empty:
    display_df = evaluation_period_df.tail(display_df_end_limit)
else:
    display_df = future_df_template.tail(display_df_end_limit)
    
cols_to_display_final = ['ds', 'y_original'] + [m[1] for m in models_to_evaluate.values() if m[1] in display_df.columns]
print(display_df[cols_to_display_final])

## 18. Conclusion and Model Selection

**Explanation:**  
Based on the evaluation metrics and model outputs, we can now draw final conclusions and make informed recommendations about model selection.

---

### 📊 Results Summary (User-Provided Metrics):

#### ✅ Prophet Model Only (P):
- **RMSE**: 736.90  
- **MAE**: 349.42

#### ✅ Standalone LSTM Model Only (SL):
- **RMSE**: 191.33  
- **MAE**: 158.62

#### ✅ Combined Model (Prophet + Residual LSTM - P + P_RL):  
(*`prophet_plus_p_rl_pred_orig` in notebook*)
- **RMSE**: 736.90  
- **MAE**: 349.42

#### ✅ Combined Model (Standalone LSTM + Residual LSTM via BMA - SL + SL_RL):  
(*`standalone_lstm_plus_sl_rl_pred_orig`, user's best BMA-weighted hybrid*)
- **RMSE**: 173.46  
- **MAE**: 144.82  
- **BMA Weights Used**: `SL = 0.9097`, `SL+RP = 0.0903`

#### ✅ BMA (Notebook's Demonstration Weights):
- **RMSE**: *(See `results_summary` output in Section 17)*  
- **MAE**: *(See `results_summary` output in Section 17)*

> *Note: Validation metrics for `P_RL` (e.g., RMSE: 0.0109, MAE: 0.0088) are from log-scale residuals and not directly comparable to original-scale forecast evaluations.*

---

### 🔎 Analysis:

- **Standalone LSTM (SL)** clearly outperforms **Prophet**, with a significantly lower RMSE.
- **Hybrid SL + SL_RL** (Standalone LSTM plus its residual-correcting LSTM) further improves performance.
- The **BMA ensemble**, when **heavily weighted toward SL and SL_RL**, produces the **best overall result**.
- **Hybrid Prophet + P_RL** offers **no improvement** over Prophet alone, indicating LSTM residual correction may not effectively address Prophet's forecasting errors in this case.

---

### 📝 Recommendations:

#### ✅ **Primary Model Candidate:**
- **Best Performing Model**:  
  **Combined SL + SL_RL**, with an RMSE of 173.46 and MAE of 144.82.
- **Corresponds to**:  
  `standalone_lstm_plus_sl_rl_pred_orig` in the notebook.
- **Supported by BMA Weights**:  
  `SL = 0.9097`, `SL+RP = 0.0903`

#### ⚙️ **Optimize BMA Weights**:
- The BMA implementation currently uses **illustrative weights**.
- Future versions should optimize these weights using:
  - Validation set performance
  - Bayesian techniques
  - Grid search or optimization libraries

#### 🔍 **Code and Pipeline Review**:
- Ensure **scaling/transformations** are applied consistently (fit on training only, transform on test).
- Validate **input alignment** across all models.
- Review external scripts or reporting tools for **typos** (e.g., `bма_weight_P`) in model identifiers or metric labels.

---

### ✅ General Summary:

This notebook demonstrates a **robust, multi-model time series forecasting pipeline** that compares:

- Classical statistical methods (Prophet)
- Neural forecasting (LSTM)
- Residual-correcting hybrids (P_RL, SL_RL)
- Ensemble methods (BMA)

The key takeaway is the **effectiveness of LSTM models**, particularly when enhanced with **residual modeling and ensemble averaging**.

The final recommended model — **SL + SL_RL hybrid with optimized BMA weights** — delivers the most accurate predictions on this submission volume dataset and provides a strong foundation for future production use or further tuning.


Explanation:
This section prepares and exports the historical actual submission volumes and the forecasted volumes from the best performing model (standalone_lstm_plus_sl_rl_pred_orig based on the conclusion) to a CSV file. This file can then be used for reporting or visualization in tools like Power BI. The forecast covers the future period defined by future_df_template.

In [ ]:
from IPython.display import display
import ipywidgets as widgets
import os # For path operations if needed in on_export_button_clicked
from datetime import datetime # Not strictly needed if using pd.Timestamp correctly
import pandas as pd # Assuming pandas is used, ensure it's imported
import numpy as np  # Assuming numpy is used, ensure it's imported

# --- Export Historical Data and Next Month Forecast to CSV ---
print("\n--- Exporting Historical and Next Month Forecast to CSV ---")

# Define chosen forecast column (log and original scale)
chosen_forecast_col_log = 'standalone_lstm_plus_sl_rl_pred_log'
chosen_forecast_col_orig = 'standalone_lstm_plus_sl_rl_pred_orig'

if chosen_forecast_col_log not in future_df_template.columns:
    print(f"Warning: Chosen forecast column '{chosen_forecast_col_log}' not found. Falling back to 'bma_pred_log'.")
    chosen_forecast_col_log = 'bma_pred_log'
    chosen_forecast_col_orig = 'bma_pred_orig'
    if chosen_forecast_col_log not in future_df_template.columns:
        raise ValueError(f"Fallback forecast column '{chosen_forecast_col_log}' also not found. Cannot export.")

# Widgets for file picker and export button
file_picker = widgets.Text(
    value='submission_volume_full_forecast.csv', # Default file name
    description='CSV File Path:',
    placeholder='Enter file path or name...',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='60%')
)
export_button = widgets.Button(description="Export Full Report to CSV")
output_area = widgets.Output()

def on_export_button_clicked(b):
    with output_area:
        output_area.clear_output() # Clear previous messages
        print("Preparing data for export...")
        try:
            # Part 1: Historical data and existing forecasts from future_df_template
            # This covers data up to eval_end_common (e.g., Feb 2025)
            part1_df = future_df_template[['ds', 'y_original', chosen_forecast_col_orig, 'avg_package_count', 
                                           'standalone_lstm_pred_log', 'sl_rl_predicted_residual_log']].copy()
            part1_df.rename(columns={chosen_forecast_col_orig: 'ForecastedVolume'}, inplace=True)
            
            # Determine type for part1_df
            part1_df['Type'] = np.where(part1_df['y_original'].notnull(), 'Historical', 'Existing Forecast')

            # Part 2: Generate new 1-month forecast (e.g., Mar 2025)
            print("Generating new 1-month forecast extension...")
            last_date_in_template = future_df_template['ds'].max() 
            new_forecast_start_ds = last_date_in_template + pd.DateOffset(months=1)
            
            num_forecast_months = 1 # Number of months to forecast
            new_forecast_dates = pd.date_range(start=new_forecast_start_ds, periods=num_forecast_months, freq='M')

            # Prepare avg_package_count for these new dates (ffill from last known)
            # Ensure 'monthly_df' and 'lookback' are defined in your notebook environment
            last_known_avg_pkg_count = monthly_df['avg_package_count'].iloc[-1]
            new_avg_pkg_counts = pd.Series([last_known_avg_pkg_count] * num_forecast_months)
            
            df_new_forecast_input = pd.DataFrame({'ds': new_forecast_dates, 'avg_package_count': new_avg_pkg_counts})

            # --- Generate SL predictions for new forecast period ---
            new_sl_preds_log_extended = []
            last_sl_log_preds_for_input = future_df_template['standalone_lstm_pred_log'].iloc[-lookback:].values
            current_sl_sequence_scaled = scaler_standalone_lstm_y.transform(last_sl_log_preds_for_input.reshape(-1, 1)).reshape(1, lookback, 1)

            for _ in range(num_forecast_months):
                pred_scaled_sl = model_standalone_lstm.predict(current_sl_sequence_scaled, verbose=0)[0,0]
                pred_log_sl = scaler_standalone_lstm_y.inverse_transform([[pred_scaled_sl]])[0,0]
                new_sl_preds_log_extended.append(pred_log_sl)
                # Update sequence for next prediction if num_forecast_months > 1
                if num_forecast_months > 1 or _ < num_forecast_months -1 : # only append if there are more steps or it's not the last step of a multi-step forecast
                     current_sl_sequence_scaled = np.append(current_sl_sequence_scaled[:, 1:, :], [[[pred_scaled_sl]]], axis=1)

            df_new_forecast_input['standalone_lstm_pred_log_extended'] = new_sl_preds_log_extended

            # --- Generate SL_RL residual predictions for new forecast period ---
            new_sl_rl_resid_preds_log_extended = []
            last_sl_rl_log_resids_for_input = future_df_template['sl_rl_predicted_residual_log'].iloc[-lookback:].values
            current_sl_rl_sequence_scaled = scaler_sl_rl_residual.transform(last_sl_rl_log_resids_for_input.reshape(-1, 1)).reshape(1, lookback, 1)

            for _ in range(num_forecast_months):
                pred_scaled_sl_rl_resid = model_sl_rl.predict(current_sl_rl_sequence_scaled, verbose=0)[0,0]
                pred_log_sl_rl_resid = scaler_sl_rl_residual.inverse_transform([[pred_scaled_sl_rl_resid]])[0,0]
                new_sl_rl_resid_preds_log_extended.append(pred_log_sl_rl_resid)
                # Update sequence for next prediction if num_forecast_months > 1
                if num_forecast_months > 1 or _ < num_forecast_months -1 : # only append if there are more steps or it's not the last step of a multi-step forecast
                    current_sl_rl_sequence_scaled = np.append(current_sl_rl_sequence_scaled[:, 1:, :], [[[pred_scaled_sl_rl_resid]]], axis=1)
            
            df_new_forecast_input['sl_rl_predicted_residual_log_extended'] = new_sl_rl_resid_preds_log_extended
            
            # Combine extended predictions
            df_new_forecast_input['ForecastedVolume_log'] = df_new_forecast_input['standalone_lstm_pred_log_extended'] + df_new_forecast_input['sl_rl_predicted_residual_log_extended']
            df_new_forecast_input['ForecastedVolume'] = np.expm1(df_new_forecast_input['ForecastedVolume_log'])
            df_new_forecast_input['y_original'] = np.nan # Actuals are NaN for new forecasts
            df_new_forecast_input['Type'] = f'New {num_forecast_months}M Forecast' # Updated type string
            
            df_new_1m_export = df_new_forecast_input[['ds', 'y_original', 'ForecastedVolume', 'Type']]

            # Combine all parts
            final_export_df = pd.concat([
                part1_df[['ds', 'y_original', 'ForecastedVolume', 'Type']], 
                df_new_1m_export
            ], ignore_index=True)
            
            final_export_df.rename(columns={
                'ds': 'Date',
                'y_original': 'ActualVolume'
            }, inplace=True)

            # Ensure ForecastedVolume is integer (whole number) and handles NaNs from historical actuals appropriately
            final_export_df['ForecastedVolume'] = final_export_df['ForecastedVolume'].round().astype('Int64')
            final_export_df['ActualVolume'] = final_export_df['ActualVolume'].astype('Int64')


            file_path = file_picker.value.strip()
            if not file_path:
                raise ValueError("Please specify a file path for export.")
            
            dir_name = os.path.dirname(file_path)
            if dir_name and not os.path.exists(dir_name): 
                os.makedirs(dir_name)
                print(f"Created directory: {dir_name}")
            
            final_export_df.to_csv(file_path, index=False, date_format='%Y-%m-%d')
            print(f"Successfully exported data to '{file_path}'")
            print("Columns in exported CSV: ", final_export_df.columns.tolist())
            print("Sample of exported data (last 15 rows):")
            print(final_export_df.tail(15))

        except Exception as e:
            print(f"Error during export: {e}")

export_button.on_click(on_export_button_clicked)
display(file_picker, export_button, output_area)

print(f"\nSet the file path above and click '{export_button.description}' to save the CSV report.")
print("The report will include historical data, model fit, and a new 1-month forecast (e.g., Mar 2025).")


In [ ]:
print("\n--- Generating New 12-Month Iterative Forecast (Mar 2025 - Feb 2026) ---")

# Determine the starting point for the new 12-month forecast
last_date_in_template = future_df_template['ds'].max()  # This is eval_end_common (2025-02-28)
new_forecast_start_ds = last_date_in_template + pd.DateOffset(months=1)
new_12m_dates = pd.date_range(start=new_forecast_start_ds, periods=12, freq='M')

print(f"Generating 12 new monthly forecasts from {new_12m_dates.min().date()} to {new_12m_dates.max().date()}")

# DataFrame to store the new 12-month forecast
df_extended_forecast = pd.DataFrame({'ds': new_12m_dates})

# --- 1. Standalone LSTM (SL) Component: Generate 12 new iterative predictions ---
new_sl_preds_log_extended = []

# Initial input sequence for SL model:
# Last 'lookback' values of 'standalone_lstm_pred_log' from future_df_template, scaled.
last_sl_log_preds_for_input = future_df_template['standalone_lstm_pred_log'].iloc[-lookback:].values
current_sl_sequence_scaled = scaler_standalone_lstm_y.transform(last_sl_log_preds_for_input.reshape(-1, 1)).reshape(1, lookback, 1)

for i in range(12):
    pred_scaled_sl = model_standalone_lstm.predict(current_sl_sequence_scaled, verbose=0)[0, 0]
    pred_log_sl = scaler_standalone_lstm_y.inverse_transform([[pred_scaled_sl]])[0, 0]
    new_sl_preds_log_extended.append(pred_log_sl)
    # Update sequence for next prediction
    current_sl_sequence_scaled = np.append(current_sl_sequence_scaled[:, 1:, :], [[[pred_scaled_sl]]], axis=1)

df_extended_forecast['sl_pred_log_extended'] = new_sl_preds_log_extended

# --- 2. Residual LSTM (SL_RL) Component: Generate 12 new iterative predictions ---
new_sl_rl_resid_preds_log_extended = []

# Initial input sequence for SL_RL model:
# Last 'lookback' values of 'sl_rl_predicted_residual_log' from future_df_template, scaled.
# This column was created in Section 14.
last_sl_rl_log_resids_for_input = future_df_template['sl_rl_predicted_residual_log'].iloc[-lookback:].values
current_sl_rl_sequence_scaled = scaler_sl_rl_residual.transform(last_sl_rl_log_resids_for_input.reshape(-1, 1)).reshape(1, lookback, 1)

for i in range(12):
    pred_scaled_sl_rl_resid = model_sl_rl.predict(current_sl_rl_sequence_scaled, verbose=0)[0, 0]
    pred_log_sl_rl_resid = scaler_sl_rl_residual.inverse_transform([[pred_scaled_sl_rl_resid]])[0, 0]
    new_sl_rl_resid_preds_log_extended.append(pred_log_sl_rl_resid)
    # Update sequence for next prediction
    current_sl_rl_sequence_scaled = np.append(current_sl_rl_sequence_scaled[:, 1:, :], [[[pred_scaled_sl_rl_resid]]], axis=1)

df_extended_forecast['sl_rl_resid_pred_log_extended'] = new_sl_rl_resid_preds_log_extended

# --- 3. Combine and Finalize New 12-Month Forecast ---
df_extended_forecast['final_extended_pred_log'] = df_extended_forecast['sl_pred_log_extended'] + df_extended_forecast['sl_rl_resid_pred_log_extended']
df_extended_forecast['ForecastedVolume_SL_plus_SL_RL'] = np.expm1(df_extended_forecast['final_extended_pred_log'])
df_extended_forecast['ForecastedVolume_SL_plus_SL_RL'] = df_extended_forecast['ForecastedVolume_SL_plus_SL_RL'].round().astype('Int64') # Whole numbers

print("\n--- New 12-Month Forecast (March 2025 - February 2026) ---")
print(df_extended_forecast[['ds', 'ForecastedVolume_SL_plus_SL_RL']])

# This df_extended_forecast can now be appended to the historical/existing forecast data for a complete timeline if needed.
# For example, for the CSV export in Section 19, this logic is integrated into the button's callback.
